In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pathlib
import pandas as pd
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold,StratifiedKFold

In [3]:
DATA_DIR = pathlib.Path(".")
DATA_FILE = "sc2021_train_deals.csv"
AGG_COLS = ["material_code", "company_code", "country", "region","manager_code"]
RS = 82736

# Загрузка данных

In [5]:
data = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), parse_dates=["month", "date"])

In [6]:
data.head()

,material_code,company_code,country,region,manager_code,month,material_lvl1_name,material_lvl2_name,material_lvl3_name,contract_type,date,volume
0,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-01,43.0
1,197,0,Китай,Китай,16350,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,95.0
2,794,2162,Казахстан,Атырауская обл.,10942,2018-01-01,Базовые полимеры,ПП,ПП,Контракт,2018-01-02,57.0
3,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,21.0
4,133,0,Китай,Китай,17745,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,150.0


In [7]:
exdata = pd.read_csv('exdata.csv', parse_dates=["month"],index_col='month')
exdata.head()

,CNYCB_mean,CNYCB_mean_d,CNYCB_std,RUBEUR_mean,RUBEUR_mean_d,RUBEUR_std,RUBUSD_mean,RUBUSD_mean_d,RUBUSD_std,USDCNY_mean,USDCNY_mean_d,USDCNY_std,INDUSDX_mean,INDUSDX_mean_d,INDUSDX_std,ICE.BRN_mean,ICE.BRN_mean_d,ICE.BRN_std
month,,,,,,,,,,,,,,,,,,
2018-10-01,9.498966,0.000000,0.077915,0.013164,0.000000,0.000146,0.015138,0.000000,0.000118,6.914600,0.000000,0.031972,95.485370,0.000000,0.566365,80.926296,0.000000,3.122605
2018-11-01,9.559282,0.060316,0.098226,0.013189,0.000025,0.000118,0.014994,-0.000144,0.000173,6.938037,0.023437,0.019812,96.592500,1.107130,0.407432,66.890769,-14.035527,5.105077
2018-12-01,9.775204,0.215921,0.172860,0.012989,-0.000200,0.000261,0.014776,-0.000218,0.000257,6.893154,-0.044883,0.027713,96.641800,0.049300,0.421880,58.099600,-8.791169,3.472374
2019-01-01,9.826233,0.051029,0.068107,0.013062,0.000073,0.000221,0.014919,0.000143,0.000233,6.793228,-0.099926,0.049816,95.490667,-1.151133,0.386490,59.930769,1.831169,2.384797
2019-02-01,9.764557,-0.061676,0.045353,0.013355,0.000293,0.000063,0.015167,0.000248,0.000084,6.737527,-0.055701,0.029396,96.435500,0.944833,0.313939,64.319583,4.388814,2.189764


In [9]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0).apply(lambda x:np.log10(x+1))

In [10]:
group_ts['_id'] = list(range(len(group_ts)))
group_ts.head()

month                                                                2018-01-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.532754   
133           0            Белоруссия Минская обл.     10942                    0.000000   
                                      Могилевская обл. 10942                    0.000000   
                                      г. Минск         10942                    0.000000   
                           Казахстан  г. Нур-Султан    13301                    0.000000   

month                                                                2018-02-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.532754   
133           0            Белоруссия Минская обл.     10942                    0.000000   
                                      Могилевская обл. 10942                    0.000000   
                                      г. Минск         10942                    1.322219   
                           Казахстан  г. Нур-Султан    13301                    0.000000   

month                                                                2018-03-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.416641   
133           0            Белоруссия Минская обл.     10942                    0.000000   
                                      Могилевская обл. 10942                    0.000000   
                                      г. Минск         10942                    0.000000   
                           Казахстан  г. Нур-Султан    13301                    1.491362   

month                                                                2018-04-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.382017   
133           0            Белоруссия Минская обл.     10942                    2.303196   
                                      Могилевская обл. 10942                    0.000000   
                                      г. Минск         10942                    0.000000   
                           Казахстан  г. Нур-Султан    13301                    1.491362   

month                                                                2018-05-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.344392   
133           0            Белоруссия Минская обл.     10942                    1.785330   
                                      Могилевская обл. 10942                    2.149219   
                                      г. Минск         10942                    1.612784   
                           Казахстан  г. Нур-Султан    13301                    0.000000   

month                                                                2018-06-01 00:00:00  \
material_code company_code country    region           manager_code                        
124           7278         Россия     Респ. Татарстан  17460                    2.344392   
133           0            Белоруссия Минская обл.     10942                    0.000000   
                                      Могилевская обл. 10942                    0.000000   
                                      г. Минск         10942                    2.206826   
                           Казахстан  г. Нур-Султан    13301                    0.000000   

month                                                                2018-07-01 00:00:00  \
material_code company_code country    

Считаем тренд объемов сделок по месяцам (во сколько раз в конце месяца больше чем в начале)

In [11]:
group_day = data.copy().groupby(AGG_COLS + ["date"])["volume"].sum().unstack(fill_value=0).stack().rename('vol')
group_day = group_day.reset_index(level=-1)
group_day['month'] = group_day.date.apply(lambda x:x.replace(day=1))
group_day['day'] = group_day.date.apply(lambda x:x.day)
group_day['max_day'] = group_day.groupby(['month'])['day'].transform('max')
group_day['max_vol'] = group_day.groupby(['month'])['vol'].transform('max')
group_day['vol'] /= group_day['max_vol']
group_day['inv_day'] = group_day['max_day']-group_day['day']+1
group_day['v'] = group_day['vol']*(group_day['day']-group_day['inv_day'])

group_ts2 = group_day.reset_index().groupby(AGG_COLS+['month'])['v'].sum().unstack(fill_value=0)

Генерация df

Таргетом является изменение значения к предыдущему месяцу

In [24]:
def get_features(df: pd.DataFrame,df2: pd.DataFrame,exdata: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    n_mounths = 12
    start_period = month - pd.offsets.MonthBegin(n_mounths)
    end_period = month - pd.offsets.MonthBegin(1)
   
    _df = df.copy()
    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    cols_months = [f"vol_tm{i}" for i in range(n_mounths, 0, -1)]
    features[cols_months] = df.loc[:, start_period:end_period].copy()
   
    #Среднее значение таргета за все предыдушие периоды
    features['_mean'] = df.mean(axis=1)
   
    for m in cols_months:
        features[m] = features[m]/features['_mean']
    features[[f"dev_tm"]] = (df2[end_period].copy()/features['vol_tm1']).fillna(0)
    
    for c in exdata.columns:
        features[c] = exdata.loc[end_period,c]
    
    for i in range(n_mounths//3, 0, -1):
        features[f"delta_vol_tm{i}"] = features[f'vol_tm{i}']-features[f'vol_tm{i+1}']
   
    features[f"delta2"] = features[f"delta_vol_tm1"]-features[f"delta_vol_tm2"]
    
    #Различные метрики за 12,6,3 месяцев
    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_year_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_year_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_year_var"))
    
    rolling = df.rolling(6, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_half_year_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_half_year_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_half_year_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_half_year_var"))
    
    rolling = df.rolling(3, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_quarter_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_quarter_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_quarter_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_quarter_var"))
    
    
    #Экстраполяции на следующий период
    features['e1'] = features[f'vol_tm2']+(features[f'vol_tm1']-features[f'vol_tm2'])*2
    features['e2'] = features[f'vol_tm1']+(features[f'vol_tm1']-features[f'vol_tm2'])
    
    #Количество 0 за последний год
    features['count0_last_year'] = df[df==0].iloc[:,-12:].isna().sum(axis=1)
    
    #Считаем измение суммарного объема за периоды 3,6,9,12 месяцев и ренд изменений
    for n in [3,6,9,12]:
        rolling = df.rolling(n, axis=1, min_periods=n-1)
        cur = rolling.sum().iloc[:, -1]
        prev = rolling.sum().iloc[:, -2]
        features[f"delta_sum_months_{n}_cur"] = cur-prev - features[f'vol_tm1']
        features[f"dev_sum_months_{n}"] = df.iloc[:,-n:].mul(np.linspace(0,1,n)-np.linspace(1,0,n),axis=1).sum(axis=1)
        
    features["month"] = month.month
    
    #Относительные метрики за прошлый и позапрошлые месяцы
    for i in range(2, 0, -1):
        for c in ['last_half_year_avg','last_half_year_max',
                  'last_quarter_avg','last_quarter_max']:
            features[f'{c}_{i}_delta'] = features[c] - features[f"vol_tm{i}"] 
            

    features['target'] = _df[month]- _df[end_period]
    
    features['_id'] = _df['_id']
    features['_target'] = _df[month]
    features['_vol_tm1'] =  _df[end_period]

    
    features = features.reset_index()
    
    return features

In [16]:
tr_range = pd.date_range("2019-01-01", "2020-07-01", freq="MS")

In [17]:
from tqdm.auto import tqdm

In [19]:
%%time
full_features = None


dataset_features = []
for target_month in tqdm(tr_range):
    features = get_features(group_ts,group_ts2,exdata, target_month)
    dataset_features.append(features)
full_features = pd.concat(dataset_features, ignore_index=True)


Wall time: 12.8 s


In [20]:
full_features.head(5)

,material_code,company_code,country,region,manager_code,month,vol_tm12,vol_tm11,vol_tm10,vol_tm9,...,last_quarter_avg_2_delta,last_quarter_max_2_delta,last_half_year_avg_1_delta,last_half_year_max_1_delta,last_quarter_avg_1_delta,last_quarter_max_1_delta,target,_id,_target,_vol_tm1
0,124,7278,Россия,Респ. Татарстан,17460,1,1.050491,1.050491,1.002331,0.987970,...,1.433076e+00,1.433076,1.380919e+00,1.433076,1.433076e+00,1.433076,-0.145510,0,2.303196,2.448706
1,133,0,Белоруссия,Минская обл.,10942,1,0.000000,0.000000,0.000000,6.759980,...,1.480297e-16,0.000000,7.401487e-17,0.000000,1.480297e-16,0.000000,0.000000,1,0.000000,0.000000
2,133,0,Белоруссия,Могилевская обл.,10942,1,0.000000,0.000000,0.000000,0.000000,...,-8.067377e-01,0.315451,9.451555e-01,2.344392,1.222204e+00,2.344392,0.000000,2,0.000000,0.000000
3,133,0,Белоруссия,г. Минск,10942,1,0.000000,0.922855,0.000000,0.000000,...,5.490175e-01,1.149446,1.085671e+00,1.680289,1.079861e+00,1.680289,0.290565,3,1.612784,1.322219
4,133,0,Казахстан,г. Нур-Султан,13301,1,0.000000,0.000000,1.682832,1.682832,...,1.034715e+00,1.612784,-5.445211e-01,-0.207059,-7.851280e-01,-0.207059,0.000000,4,1.612784,1.612784


## Тренировка

In [22]:
CAT_COLS = ['month']+AGG_COLS
FTS_COLS = full_features.columns.drop(['target','_id','_target', '_vol_tm1'])

TARGET = "target"
FTS_COLS

Index(['material_code', 'company_code', 'country', 'region', 'manager_code',
       'month', 'vol_tm12', 'vol_tm11', 'vol_tm10', 'vol_tm9', 'vol_tm8',
       'vol_tm7', 'vol_tm6', 'vol_tm5', 'vol_tm4', 'vol_tm3', 'vol_tm2',
       'vol_tm1', '_mean', 'dev_tm', 'CNYCB_mean', 'CNYCB_mean_d', 'CNYCB_std',
       'RUBEUR_mean', 'RUBEUR_mean_d', 'RUBEUR_std', 'RUBUSD_mean',
       'RUBUSD_mean_d', 'RUBUSD_std', 'USDCNY_mean', 'USDCNY_mean_d',
       'USDCNY_std', 'INDUSDX_mean', 'INDUSDX_mean_d', 'INDUSDX_std',
       'ICE.BRN_mean', 'ICE.BRN_mean_d', 'ICE.BRN_std', 'delta_vol_tm4',
       'delta_vol_tm3', 'delta_vol_tm2', 'delta_vol_tm1', 'delta2',
       'last_year_avg', 'last_year_max', 'last_year_std', 'last_year_var',
       'last_half_year_avg', 'last_half_year_max', 'last_half_year_std',
       'last_half_year_var', 'last_quarter_avg', 'last_quarter_max',
       'last_quarter_std', 'last_quarter_var', 'e1', 'e2', 'count0_last_year',
       'delta_sum_months_3_cur', 'dev_sum_months_3'

In [879]:
params = {'learning_rate': 0.010275773251508875, 'depth': 11, 'l2_leaf_reg': 0.10438262443505627}

In [894]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RS)
preds = []

for fold,(train_idx, val_idx) in enumerate(kf.split(full_features,full_features['_id'])):

    train = full_features.iloc[train_idx]
    val = full_features.iloc[val_idx]
    model = CatBoostRegressor(iterations=10000,
                              early_stopping_rounds=300,
                              cat_features=CAT_COLS,
                              random_state=RS*fold,
                              verbose=100,
                              loss_function='RMSE',
                              **params
                         )
    print(f"########### Start FOLD {fold} ##########")
    model.fit(train[FTS_COLS], train[TARGET],
              eval_set=(val[FTS_COLS], val[TARGET]))
    
    
    pred = model.predict(val[FTS_COLS])
    
    y_true = val['_target']
    y_pred = (val['_vol_tm1']+np.array(pred)).clip(0,100)
   
    
    print("FOLD",fold," = "
      f'{np.sqrt(mean_squared_log_error(10**(y_true)-1, 10**(y_pred)-1)):.4f}')
          
    
    print('')
    
    model.save_model(f"model{fold}.cbm")
    
    


    
#2729

########### Start FOLD 0 ##########


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.8136941	test: 0.8294845	best: 0.8294845 (0)	total: 473ms	remaining: 1h 18m 53s
100:	learn: 0.6670789	test: 0.7213854	best: 0.7213854 (100)	total: 45.1s	remaining: 1h 13m 44s
200:	learn: 0.6068115	test: 0.7009591	best: 0.7009591 (200)	total: 1m 30s	remaining: 1h 13m 17s
300:	learn: 0.5601914	test: 0.6942509	best: 0.6942509 (300)	total: 2m 15s	remaining: 1h 12m 34s
400:	learn: 0.5254752	test: 0.6920912	best: 0.6920912 (400)	total: 3m	remaining: 1h 11m 52s
500:	learn: 0.4968758	test: 0.6904453	best: 0.6904083 (496)	total: 3m 52s	remaining: 1h 13m 35s
600:	learn: 0.4736033	test: 0.6893612	best: 0.6893612 (600)	total: 4m 39s	remaining: 1h 12m 57s
700:	learn: 0.4518363	test: 0.6889083	best: 0.6888843 (699)	total: 5m 24s	remaining: 1h 11m 40s
800:	learn: 0.4311564	test: 0.6882344	best: 0.6882283 (799)	total: 6m 8s	remaining: 1h 10m 34s
900:	learn: 0.4131237	test: 0.6876343	best: 0.6876343 (900)	total: 6m 54s	remaining: 1h 9m 49s
1000:	learn: 0.3973660	test: 0.6872456	best: 0.68722

900:	learn: 0.4090194	test: 0.6838160	best: 0.6838000 (897)	total: 6m 41s	remaining: 1h 7m 32s
1000:	learn: 0.3915534	test: 0.6836113	best: 0.6836113 (1000)	total: 7m 26s	remaining: 1h 6m 53s
1100:	learn: 0.3748562	test: 0.6833449	best: 0.6832736 (1086)	total: 8m 11s	remaining: 1h 6m 14s
1200:	learn: 0.3588165	test: 0.6830134	best: 0.6830134 (1200)	total: 8m 57s	remaining: 1h 5m 36s
1300:	learn: 0.3425583	test: 0.6826501	best: 0.6826364 (1276)	total: 9m 42s	remaining: 1h 4m 56s
1400:	learn: 0.3290046	test: 0.6825781	best: 0.6825394 (1384)	total: 10m 28s	remaining: 1h 4m 15s
1500:	learn: 0.3160826	test: 0.6827841	best: 0.6825394 (1384)	total: 11m 13s	remaining: 1h 3m 33s
1600:	learn: 0.3033231	test: 0.6826991	best: 0.6825394 (1384)	total: 11m 58s	remaining: 1h 2m 50s
1700:	learn: 0.2916783	test: 0.6823829	best: 0.6823549 (1695)	total: 12m 45s	remaining: 1h 2m 15s
1800:	learn: 0.2802028	test: 0.6823650	best: 0.6822816 (1741)	total: 13m 31s	remaining: 1h 1m 33s
1900:	learn: 0.2697331	test

1400:	learn: 0.3265136	test: 0.6725527	best: 0.6725428 (1393)	total: 10m 35s	remaining: 1h 5m 1s
1500:	learn: 0.3143698	test: 0.6727176	best: 0.6724886 (1420)	total: 11m 21s	remaining: 1h 4m 16s
1600:	learn: 0.3026724	test: 0.6726834	best: 0.6724886 (1420)	total: 12m 7s	remaining: 1h 3m 34s
1700:	learn: 0.2908910	test: 0.6725484	best: 0.6724886 (1420)	total: 12m 52s	remaining: 1h 2m 49s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.6724886155
bestIteration = 1420

Shrink model to first 1421 iterations.
FOLD 4  = 1.5483
FOLD Val 4  = 1.1702



# Формирование загрузки

Результатом является усредненное значение 5 моделей, после np.floor для уничтожение "нечетких" нолей

In [34]:
%%writefile predict.py

import pathlib
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import pickle

MODEL_FILE = pathlib.Path(__file__).parent.joinpath("baseline_model.cbm")
EXDATA_FILE = pathlib.Path(__file__).parent.joinpath("exdata.csv")
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]

FOLDS = 5

def get_features(df: pd.DataFrame,df_count: pd.DataFrame,df2: pd.DataFrame,exdata: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    n_mounths = 12
    start_period = month - pd.offsets.MonthBegin(n_mounths)
    end_period = month - pd.offsets.MonthBegin(1)
   
    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    cols_months = [f"vol_tm{i}" for i in range(n_mounths, 0, -1)]
    features[cols_months] = df.loc[:, start_period:end_period].copy()
   
    features['_mean'] = df.mean(axis=1)
   
    for m in cols_months:
        features[m] = features[m]/features['_mean']
    features[[f"dev_tm"]] = (df2[end_period].copy()/features['vol_tm1']).fillna(0)
    
    for c in exdata.columns:
        features[c] = exdata.loc[end_period,c]
    
    for i in range(n_mounths//3, 0, -1):
        features[f"delta_vol_tm{i}"] = features[f'vol_tm{i}']-features[f'vol_tm{i+1}']
   
    features[f"delta2"] = features[f"delta_vol_tm1"]-features[f"delta_vol_tm2"]
    
    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_year_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_year_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_year_var"))
    
    rolling = df.rolling(6, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_half_year_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_half_year_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_half_year_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_half_year_var"))
    
    rolling = df.rolling(3, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_quarter_avg"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_quarter_max"))
    features = features.join(rolling.std().iloc[:, -1].rename("last_quarter_std"))
    features = features.join(rolling.var().iloc[:, -1].rename("last_quarter_var"))
    
    

    features['e1'] = features[f'vol_tm2']+(features[f'vol_tm1']-features[f'vol_tm2'])*2
    features['e2'] = features[f'vol_tm1']+(features[f'vol_tm1']-features[f'vol_tm2'])
    

    features['count0_last_year'] = df[df==0].iloc[:,-12:].isna().sum(axis=1)
    
    for n in [3,6,9,12]:
        rolling = df.rolling(n, axis=1, min_periods=n-1)
        cur = rolling.sum().iloc[:, -1]
        prev = rolling.sum().iloc[:, -2]
        features[f"delta_sum_months_{n}_cur"] = cur-prev - features[f'vol_tm1']
        features[f"dev_sum_months_{n}"] = df.iloc[:,-n:].mul(np.linspace(0,1,n)-np.linspace(1,0,n),axis=1).sum(axis=1)
        
    features["month"] = month.month
    
    for i in range(2, 0, -1):
        for c in ['last_half_year_avg','last_half_year_max',
                  'last_quarter_avg','last_quarter_max']:
            features[f'{c}_{i}_delta'] = features[c] - features[f"vol_tm{i}"] 
            
    features['_vol_tm1'] =  df[end_period]
    
    return features.reset_index()


def predict(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    group_ts = df.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0).apply(lambda x:np.log10(x+1))
    group_ts_count = df.groupby(AGG_COLS + ["month"])["volume"].count().unstack(fill_value=0).apply(lambda x:np.log10(x+1))
    
    group_day = df.copy().groupby(AGG_COLS + ["date"])["volume"].sum().unstack(fill_value=0).stack().rename('vol')
    group_day = group_day.reset_index(level=-1)
    group_day['month'] = group_day.date.apply(lambda x:x.replace(day=1))
    group_day['day'] = group_day.date.apply(lambda x:x.day)
    group_day['max_day'] = group_day.groupby(['month'])['day'].transform('max')
    group_day['max_vol'] = group_day.groupby(['month'])['vol'].transform('max')
    group_day['vol'] /= group_day['max_vol']
    group_day['inv_day'] = group_day['max_day']-group_day['day']+1
    group_day['v'] = group_day['vol']*(group_day['day']-group_day['inv_day'])

    group_ts2 = group_day.reset_index().groupby(AGG_COLS+['month'])['v'].sum().unstack(fill_value=0)
    
    exdata = pd.read_csv(EXDATA_FILE, parse_dates=["month"],index_col='month')
    
    features = get_features(group_ts,group_ts_count,group_ts2,exdata, month)

    model = CatBoostRegressor()
    pred = []
    for i in range(FOLDS):
        MODEL_FILE = pathlib.Path(__file__).parent.joinpath(f"model{i}.cbm")
        model.load_model(MODEL_FILE)
        predictions = model.predict(features[model.feature_names_])
        pred.append(predictions)
        
    pred = np.mean(pred,axis=0)
    
    preds_df = features[AGG_COLS].copy()
    pred = (features['_vol_tm1']+pred).clip(0,100)
    pred = 10**(pred)-1
    pred = np.floor(pred)
    preds_df["prediction"] = pred
    
    return preds_df
    

Overwriting predict.py


Протестируем:

In [35]:
import predict
import imp
imp.reload(predict)

<module 'predict' from 'C:\\jupyter\\SC2021\\sub63(53v3)\\predict.py'>

In [36]:
predict.predict(data, pd.Timestamp("2020-08-01"))

,material_code,company_code,country,region,manager_code,prediction
0,124,7278,Россия,Респ. Татарстан,17460,1.0
1,133,0,Белоруссия,Минская обл.,10942,151.0
2,133,0,Белоруссия,Могилевская обл.,10942,1.0
3,133,0,Белоруссия,г. Минск,10942,2.0
4,133,0,Казахстан,г. Нур-Султан,13301,4.0
...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,59.0
937,998,0,Россия,Ленинградская обл.,18079,3.0
938,998,3380,Россия,Ленинградская обл.,14956,34.0
939,998,5410,Россия,г. Санкт-Петербург,14956,11.0


Сохраним сабмит

In [37]:
from zipfile import ZipFile

In [38]:
zipObj = ZipFile('sub.zip', 'w')
for i in range(5):
    zipObj.write(f"model{i}.cbm")
zipObj.write('predict.py')
zipObj.write('exdata.csv')
zipObj.close()
